<a href="https://colab.research.google.com/github/RiccardoCozzi96/DeepComedy/blob/main/Generations_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generations evaluation

This notebook is provided for demostrating how we have read all the log files of all the models, extracted the generated texts for each temperature and evaluated them using the specified metrics. This step took some time (we run it on over 247 texts in almost 4 hours) because evaluating a single canto is sometimes a complex operation (depending on the used metrics). 

At the end, we generate the file "*results.csv*" in which all the scores for each generated canto are stored and accessible in an easy way in other notebooks. 

## Setup

In [1]:
!pip install pyphen
!pip install nltk --upgrade
!pip install tensorflow_datasets # used for student's metrics
import tensorflow_datasets as tfds

import os
import re
import sys
import json
import resource
import numpy as np
import pandas as pd
import pyphen
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from nltk.metrics import edit_distance


# retrieve our GitHub repository
!git clone "https://github.com/RiccardoCozzi96/DeepComedy"

sys.path.append("DeepComedy/tokenizer/")
sys.path.append("DeepComedy/metrics/")
sys.path.append("DeepComedy/datasets/")

from comedy_tokenizer import ComedyTokenizer
from comedy_metrics import *

     |████████████████████████████████| 1.9MB 2.7MB/s 
     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=5c96994f58434c77a509e701e6b0a86302da87cc835d15739e5b209dbaece2ca
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Cloning into 'DeepComedy'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 405 (delta 33), reused 0 (delta 0), pack-reused 345
Receiving objects: 100% (405/405), 1.29 MiB | 1.48 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [2]:
path = f"DeepComedy/generated cantos/"
dict_path = "DeepComedy/tokenizer/dantes_hyphenation_dictionary.csv"
comedy_filename = "DeepComedy/datasets/commedia.txt"
results_filename = "DeepComedy/results.csv"

## Functions definition

### Metric functions

In [3]:
"""imported from comedy_metrics module on the GitHub repo"""

# TABLE_WIDTH = 100 # constant for printing "fill" characters

# # returns the texts with no 
# def extract_only_verses(canto):
#   return [line for line in canto if line != "\n" and line != ""]


# def remove_accents(string):
#   string = re.sub(r"[àá]", "a", string)
#   string = re.sub(r"[èé]", "e", string)
#   string = re.sub(r"[ìí]", "i", string)
#   string = re.sub(r"[òó]", "o", string)
#   string = re.sub(r"[ùú]", "u", string)
#   return string


# def extract_vocabulary(text):
#   if type(text) == list:
#     s = set()
#     for string in text:
#       s = s.union(extract_vocabulary(string))
#     vocab = s
#     return vocab
#   else: 
#     vocab = text.lower()
#     vocab = remove_accents(vocab)
#     vocab = text.replace("\n", " ") 
#     vocab = re.sub(r"[^a-z\s]", " ", vocab)
#     all_words = vocab.split(" ")
#     unique_words = np.unique(all_words)
#     vocab = set([r for r in unique_words if len(r) > 2 ])
#     return vocab


# def count_verses(canto):
#   return len(extract_only_verses(canto))


# def evaluate_structure(canto, final_single_verse=True, verbose=False):
#   """
#   canto :\n\tlist of strings, each one representing a verse \n
#   final_single_verse (default=True):\n\tif False, return the maximum score (1.0) iff all the tercets have 3 verses. \n
#   Otherwise, the maximum score is reached iff the canto ends with a single verse \n
#   verbose (default=False): \n\tif True, prints out the count of verses, groups and tercets for each verse \n
#   Return\n\tstructuredness score in [0, 1]
#   """
#   groups = 1
#   tercets = 0
#   verse_counter = 0
#   if verbose: 
#     print("{:>40}\t{:<10}  {:<10}  {:<15}".format("", "n. verse", "n. groups", "n. tercets"))
#     print("-"*TABLE_WIDTH)

#   # count groups of verces
#   for i, verse in enumerate(canto):
#     verse = verse.replace("\n", "")
#     if not bool(re.search("[a-zA-z]", verse)):
#       groups += 1
#       if verse_counter == 3:
#         tercets += 1
#       verse_counter = 0
#     else:
#       verse_counter += 1 

#     # count the final group which is ignored
#     if i == len(canto)-1 and verse_counter == 3:
#       tercets += 1

#     if verbose: print("-{:<40}\t{:<10}  {:<10}  {:<15}".format(verse, verse_counter, groups, tercets))

#   # check whether the final verse is in a single line
#   if final_single_verse:
#     correct = (not bool(re.search("[a-zA-z]", canto[-2])) and bool(re.search("[a-zA-z]", canto[-1])))
#     if correct:
#       tercets += 1
#     else: 
#       groups += 1
#       # print("ultimo: [{}]".format(canto[-1]))
#       # print("penultimo: [{}]".format(canto[-2]))
#     if verbose: 
#       print(" "*40 + " ")
#       print("\t  single final verse:   {}\t\t{:<10}  {:<10}  {:<15}".format(correct, 
#                                                                             verse_counter, 
#                                                                             groups, 
#                                                                             tercets))  
#   if verbose: print("-"*TABLE_WIDTH) 

#   return tercets / groups



# # count number of hendecasyllables verses
# def evaluate_hendecasyllables(canto, tokenizer, return_count=False, tolerance=1, verbose=False):
#     verses = [line for line in canto if line != "\n" and line != ""]
        
#     n_syllables = []
#     for verse in [v for v in verses if v != ""]:
#         verse = verse.replace("\n", "")
#         if verse != "":
#             verse = tokenizer.remove_punctuation(verse)
#             sill, n = tokenizer.tokenize_phrase(verse, count_syllables=True)
#             if verbose: print("{:40}{:80} ({})".format(verse, sill, n))
#             n_syllables.append(n)
    
#     n_hendecasillables = len([n for n in n_syllables if n in range(11-tolerance, 11+tolerance+1)])
    
#     if verbose: 
#         print("="*80)

#     if return_count:
#         return n_hendecasillables/len(verses), n_syllables
#     else:
#         return n_hendecasillables/len(verses)



# def average_hendecasyllables(canto, tokenizer, tolerance):
#   _, n_syllables = evaluate_hendecasyllables(canto=canto, tokenizer=tokenizer, tolerance=tolerance, return_count=True)
#   return np.average(n_syllables)



# def avg_rhyming_score(canto, tokenizer, return_n_rhymes=False, raw=True, verbose=False):
#   if raw:
#     finals = []
#     verses = extract_only_verses(canto)
#     for verse in verses:
#       finals.append(tokenizer.tokenize_phrase(tokenizer.remove_punctuation(verse)).split(" ")[-2])
    
#     # find chained rhyme pattern
#     scores = []
#     for i in range(0, len(verses)-4, 3):
#       score = 0
#       if finals[i] == finals[i+2]:
#         score += 0.5
#       if finals[i+1] == finals[i+3]:
#         score += 0.5
#       scores.append(score)
#       if verbose and score != 1: print("\n{:50} ({})\n{:50} ({})\n{:50} ({})\n{:50} ({})\n- score: {}".format(
#           verses[i], finals[i], verses[i+1], finals[i+1], 
#           verses[i+2], finals[i+2], verses[i+3], finals[i+3],
#           score
#       ))


#     if return_n_rhymes:
#       return np.average(scores), len([s for s in scores if s == 1])
#     else:
#       return np.average(scores)

#   else:
#     return None


# def ngrams_plagiarism(generated_text, original_text, n=4):
#   """
#   (credits: Luga Giuliani)
#   """
#   return -1
#   # the tokenizer is used to remove non-alphanumeric symbols
#   tokenizer = tfds.features.text.Tokenizer()
#   original_text = tokenizer.join(tokenizer.tokenize(original_text.lower()))
#   generated_text_tokens = tokenizer.tokenize(generated_text.lower())

#   total_ngrams = len(generated_text_tokens) - n + 1
#   plagiarism_counter = 0

#   for i in range(total_ngrams):
#       ngram = tokenizer.join(generated_text_tokens[i:i+n])
#       plagiarism_counter += 1 if ngram in original_text else 0
#   return 1 - (plagiarism_counter / total_ngrams)


# def find_similar_words(word:str, vocabulary:set, verbose=False, return_best_distance=False):
#   """
#   Given a word, find the most similar words in the vocabulary. \n
#   The similarity between words is computed by the 'edit (Levenshtein) distance'. \n
#   If more than one word from the vocabulary has a same distance from the requested word, \n
#   then a list containing them is returned. \n
#   If #return_best_distance is True, then returns a tuple containing the best words and the best distance. 
#   """
#   if verbose: print(f"looking for words similar to '{word}'")
#   # try normal search
#   if word in vocabulary:
#     if verbose: print(f"\t{(real_word, 0)} <-- match")
#     return ([word], 0) if return_best_distance else [word]

#   else:
#     most_similar = []
#     best_distance = len(word) # the distance between a word and an empty word is equal to the lenght of the word itself

#     for real_word in vocabulary:
#       dist = word_distance(word, real_word)
#       if dist <= best_distance:
#         if verbose: print(f"\t {real_word} ({dist})")
#         if dist < best_distance:
#           best_distance = dist
#           most_similar = []
#         most_similar.append(real_word)
#     return (most_similar, best_distance) if return_best_distance else most_similar


# def word_distance(a, b):
#   from nltk.metrics import edit_distance
#   d = edit_distance(a, b)
#   return d



# def incorrectness(words:set, real_words:set, verbose=False, return_match_ratio=False, plot_frequencies=False):
#   """
#   Measures the amount of incorrect words, with respect to the given set of real words. 
#   If all the passed words exists in the real words set, then the returned value is 0, otherwise return a positive real number.
#   The score is computed as a weighted average of the frequencies of the distances of the words from the real words. 
#   A set of words each of which has the nearest word (in Levenshtein distance) at 1 or 2 is way better of another one
#   whose nearest word is 10 points far from the most similar real word. 
#   """
#   if verbose: print("{}\n{:3}\t{:10}\t{:5}\t{}\n{}\n".format("="*40, "%", "WORD", "DIST", "SIMILAR TO", "="*40))
#   n_real_words = len(words)
#   # compute frequencies
#   distances = []
#   for i, my_word in enumerate(words):
#     most_similar, distance = find_similar_words(word=my_word, 
#                                                  vocabulary=real_words,
#                                                  return_best_distance=True)
#     distances.append(distance)
    
#   # compute frequencies
#   frequencies = dict(zip(np.unique(distances), [distances.count(d) for d in np.unique(distances)]))
  
#   # add the zero-frequency if not present (in order to compute the correct words percentage)
#   if 0 not in frequencies.keys(): frequencies[0] = 0
  
#   if verbose: print("\n{}\n frequencies: {}".format("-"*40, dict(frequencies)))
  
#   # computing correctness
#   incorrectness = round(np.average(np.unique(distances),
#                                  weights=[distances.count(d) for
#                                           d in np.unique(distances)]), 2)
  
#   # percentage of incorrect words
#   ratio = 1 - round(frequencies[0] / len(words), 2)

#   # print final results
#   if verbose: 
#     print(" match ratio:  {} %\t({} / {}) \n{}".format(ratio, frequencies[0],
#                                                        len(words), "="*40))
#     if distance != 0: 
#       print("{:>3}\t{:15}\t{:<5}\t{}".format(round(i/n_real_words*100, 1),
#                                              my_word, distance, most_similar))  
#   # plot
#   if plot_frequencies: 
#     from matplotlib import pyplot as pyplot
#     plt.bar(list(frequencies), [frequencies[key] for key in list(frequencies)])
#     plt.show()

#   return (incorrectness, ratio) if return_match_ratio else incorrectness

# ############################################


'imported from comedy_metrics module on the GitHub repo'

### Functions for evaluation and printing

In [4]:
def evaluate(canto, tokenizer, original_text_filename=None, verbose=False):
  
  # clean text
  canto = [verse.lower().replace("\n", "") for verse in canto]  # delete all the "\n"
  if canto[0] == "": canto = canto[1:]                          # delete the first line if it is empty
  if canto[-1] == "": canto = canto[:-1]                        # delete the last line if it is empty
  
  # compute scores
  verses_number = count_verses(canto)
  structure_score = round(evaluate_structure(canto, final_single_verse=True, verbose=verbose), 3)
  hendecasyllables_score = round(evaluate_hendecasyllables(canto, tokenizer, tolerance=1, verbose=verbose), 3)
  mean_syllables = round(1 - abs(11 - average_hendecasyllables(canto, tokenizer, tolerance=1)), 2)
  rhymeness_score = round(avg_rhyming_score(canto, tokenizer, verbose=verbose), 3)
  if original_text_filename != None:
    original_text = open(original_text_filename).read()
    incorrectness_score = round(incorrectness(extract_vocabulary(canto), extract_vocabulary(original_text)), 3)
    plagiarism_score = round(ngrams_plagiarism(''.join(canto), original_text), 3)

  return (verses_number, structure_score, hendecasyllables_score, 
          mean_syllables, rhymeness_score, incorrectness_score, plagiarism_score)






def evaluate_cantos(list_of_cantos, tokenizer, original_text_filename=None, verbose=False):
  if len(np.shape(list_of_cantos)) == 1 and type(list_of_cantos[0]) == str:
    list_of_cantos = [list_of_cantos]     # transform the single canto passed into a list of 1 canto
  elif len(np.shape(list_of_cantos)) > 2:
    raise Exception(("\nlist_of_cantos can only be:" + 
                     "\n - a single canto as a list of strings: shape (n,)" + 
                     "\n - a list of cantos as a list of lists of strings: shape (1, n)" +
                     f"\n\nlist_of_cantos passed has shape {np.shape(list_of_cantos)}"))

  results = {
      "n_vers": [],
      "struct": [],
      "hendec": [],
      "avg_syl": [],
      "rhymes": [],
      "incorr": [],
      "plagiar": []
   }

  for i,canto in enumerate(list_of_cantos):
    print(f"Evaluating canto {i+1} of {len(list_of_cantos)}...", end="")
    (verses_number, structure_score, hendecasyllables_score, 
    mean_syllables, rhymeness_score, incorrectness_score, 
    plagiarism_score) = evaluate(canto,
                                 tokenizer, 
                                 original_text_filename,
                                 verbose)
    print("done.")
    results["n_vers"].append(verses_number)
    results["struct"].append(structure_score)
    results["hendec"].append(hendecasyllables_score)
    results["avg_syl"].append(mean_syllables)
    results["rhymes"].append(rhymeness_score)
    results["incorr"].append(-incorrectness_score) # save the negative incorrectness score to have the best score as the highest value
    results["plagiar"].append(plagiarism_score)

  return results



def print_results_table(results, names):

  print("{}\n{:^100}\n{}".format("="*TABLE_WIDTH, "BEST SCORES", "="*TABLE_WIDTH))
  col_width = int(80 / len(list(results)))+1
  cell = "{:<" + str(col_width) + "}"

  # rows: metrics,   columns: cantos
  if len(names) <= 3:

    print("{:<20}".format("METRIC"), end="")

    for name in names:
      print(cell.format(name), end="")
    print("\n" + "-"*TABLE_WIDTH)

    for key in results.keys():
      print("{:<20}".format(key), end="")
      for value in results[key]:
        print(cell.format(f"[ {value} ]" if (value == max(results[key])) else
                          value), end="")
      print()
    print("\n{}".format("="*TABLE_WIDTH))


  # rows: cantos,   columns: metrics
  else:
    print("{:40}".format("CANTO"), end="")              # heading "canto"

    for colname in list(results):                       # headings of other metrics
      print(cell.format(colname), end="")
    print("\n" + "-"*TABLE_WIDTH)

    for i in range(len(names)):
      print("{:40}".format(names[i]), end="")
      for key in list(results):
        value = results[key][i]
        print(cell.format(f"[ {value} ]" if value == max(results[key]) else
                          value), end="")
      print()

### Functions for reading and exporting files

In [5]:
def read_files(dir, format="json"):                                                                                                  
  logs_list = []
  logs_names = []     
  logs = {}                                                                                              
  subdirs = [x[0] for x in os.walk(dir)]                                                                            
  for subdir in subdirs:                                                                                            
    files = os.walk(subdir).__next__()[2]                                                                             
    if (len(files) > 0):                                                                                          
      for file in files:        
        if file.endswith(f".{format}"):
          with open(os.path.join(subdir, file), 'r') as fp:
            logs[file] = json.load(fp)                                                              
  return logs


def save_as_dataframe(dictionary, file_name):
  import pandas as pd
  df = pd.DataFrame()
  info = np.array([g.split("/") for g in generations_names])
  df["model"] = [m.replace("LOG_", "") for m in info[:, 0]]
  df["temperature"] = [float(m.replace("temp_", "")) for m in info[:, 1]]
  for key in dictionary.keys():
    df[key] = dictionary[key]
  df.to_csv(file_name)
  print("\n\n--------------------\nsaved as", file_name)
  return df

## Load files

In [6]:
logs = read_files(path)
print(len(logs), "log files.\n")
for l in logs:
  print(l)

22 log files.

LOG_1_7_256_512_4_0_150.json
LOG_7_1_256_512_4_0_150.json
LOG_5_5_256_512_4_150_150.json
LOG_7_5_256_512_4_0_150.json
LOG_1_5_256_512_4_0_150.json
LOG_1_1_256_512_4_0_150.json
LOG_1_3_256_512_4_0_150.json
LOG_5_7_256_512_4_0_150.json
LOG_5_3_256_512_4_150_150.json
LOG_5_5_256_512_4_0_70.json
LOG_5_3_256_512_4_70_70.json
LOG_5_1_256_512_4_0_150.json
LOG_6_6_256_512_4_0_70.json
LOG_5_5_256_512_4_70_70.json
LOG_5_5_256_512_4_0_150.json
LOG_3_1_256_512_4_0_150.json
LOG_6_6_256_512_4_0_150.json
LOG_5_3_256_512_4_0_150.json
LOG_3_3_256_512_4_0_150.json
LOG_7_3_256_512_4_0_150.json
LOG_3_3_256_512_4_150_150.json
LOG_7_7_256_512_4_0_150.json


In [7]:
generations = []
generations_names = []
for log in logs:
  for temp in logs[log]['generations']:
    gen = log.replace('.json', '/')+temp
    generations.append(logs[log]['generations'][temp].replace(' ,',',').replace(' .','.').replace(' !','!').replace(' ?','?').replace(' :',':').replace(' ;',';').split('\n'))
    generations_names.append(gen)

print(len(generations_names), "cantos")

242 cantos


## Evaluation of cantos

### Load the hyphenation dictionary


In [8]:
dante_tokenizer = ComedyTokenizer.from_dataframe(pd.read_csv(dict_path,
                                                             index_col="word"),
                                           synalepha=True,
                                           use_tercets=True)

print("tokenizer loaded.\nhyphenation test:\t", dante_tokenizer.tokenize_phrase("una selva oscura"))

tokenizer loaded.
hyphenation test:	 <V> ù na <S> sél va~o scu ra </V>


## Evaluating generated cantos from log file


Run this cell to evaluate all the canots. 
(**NOTE**: it can be take one minute for each canto)

In [9]:
# # compute the results among all the generated cantos in the log
# print(f"Evaluating {len(generations)} cantos...")
# results = evaluate_cantos(list_of_cantos=generations, 
#                           tokenizer=dante_tokenizer, 
#                           original_text_filename=comedy_filename,
#                           verbose=False)

In [10]:
# save_as_dataframe(results, "test.csv")

In [11]:
# load results as .csv file as pandas dataframe
df_results = pd.read_csv(results_filename, index_col=0)
df_results.head()

,model,temperature,n_vers,struct,hendec,avg_syl,rhymes,incorr,plagiar
id,,,,,,,,,
0,1_1_256_512_4_0_150,0.5,100,1.0,0.87,0.65,0.891,-0.44,-1
1,1_1_256_512_4_0_150,0.6,100,1.0,0.90,0.81,0.891,-0.43,-1
2,1_1_256_512_4_0_150,0.7,100,1.0,0.90,0.70,0.875,-0.45,-1
3,1_1_256_512_4_0_150,0.8,100,1.0,0.92,0.90,0.828,-0.32,-1
4,1_1_256_512_4_0_150,0.9,100,1.0,0.90,0.81,0.891,-0.51,-1
